# Movie Rating Data ETL Project

This project is a demonstration of an ETL (Extraction, Transformation, and Loading) process of data from IMDb movie rating files to a SQL Server database. The purpose of this project is to show you how to perform these basic operations using Python tools and libraries, all directly from a Jupyter Notebook.

## Project Steps

### 1. Data Extraction

To extract the data, we used the Kaggle module. The steps are as follows:

- Install the Kaggle library: `pip install kaggle`.
- Configure your Kaggle API key.
- Use the Kaggle library to download IMDb files from Kaggle.

### 2. Data Transformation

We performed several transformations on the raw data, including:

- Filtering for movies only (removing other title types).
- Combining rating and movie details data.
- Data type conversion and column renaming.

### 3. Database Structure

We created an SQL Server database structure using SQL. The SQL script `create_schema.sql` defines the `movies` and `ratings` tables and their relationships.

### 4. Data Loading

The transformed data was loaded into the SQL Server database using the `pyodbc` library. The Python script in the Jupyter Notebook performs this operation.

## Usage

1. Open the Jupyter Notebook in which you executed the ETL steps.
2. Ensure that the `title.ratings.tsv` and `title.basics.tsv` files are in the `data/` directory.
3. Execute the code in the Jupyter Notebook to perform the ETL.
4. Check your SQL Server database for the newly inserted data.

## Notes

- Ensure that the `title.ratings.tsv` and `title.basics.tsv` files are in the `data/` directory.
- Customize the Python script in the Jupyter Notebook to use your SQL Server database information.

## Author

[Ettaki El Mehdi](https://github.com/yourgithubprofile)


### Step 1: Data Extraction

In [ ]:
# Upload kaggle.json
import pandas as pd
uploaded = pd.read_csv("C:\Users\squad\Downloads\kaggle.json")

# Move kaggle.json to the correct directory and set permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Create the directory to save the datasets
!mkdir Data

# Download multiple datasets by specifying the dataset owner and name in the list below
datasets = [
    "ashirwadsangwan/imdb-dataset"
]

for dataset in datasets:
    !kaggle datasets download -d {dataset} -p Data
    !unzip Data/{dataset.split("/")[-1]}.zip -d Data

100% 1.12G/1.12G [00:09<00:00, 180MB/s]
100% 1.12G/1.12G [00:09<00:00, 126MB/s]
Archive:  Data/imdb-dataset.zip
  inflating: Data/name.basics.tsv/data.tsv  
  inflating: Data/title.akas.tsv/data.tsv  
  inflating: Data/title.basics.tsv/data.tsv  
  inflating: Data/title.principals.tsv/data.tsv  
  inflating: Data/title.ratings.tsv/data.tsv  


In [ ]:
# Load IMDb datasets
ratings_df = pd.read_csv("/content/Data/title.ratings.tsv/data.tsv", sep='\t')
basics_df =  pd.read_csv("/content/Data/title.basics.tsv/data.tsv", sep='\t')

# Sample data exploration
ratings_df.head()

<ipython-input-15-509b2c0de580>:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df =  pd.read_csv("/content/Data/title.basics.tsv/data.tsv", sep='\t')


,tconst,averageRating,numVotes
0,tt0000001,5.7,2000
1,tt0000002,5.8,269
2,tt0000003,6.5,1889
3,tt0000004,5.5,178
4,tt0000005,6.2,2676


In [ ]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


### Step 2 : Data Transformation

In [ ]:
# Data cleaning and transformation
# Assuming we only want movie ratings for movies of type 'movie'
filtered_basics = basics_df[basics_df['titleType'] == 'movie']
merged_data = pd.merge(ratings_df, filtered_basics, on='tconst', how='inner')
merged_data.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,5.3,207,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
1,tt0000147,5.3,482,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
2,tt0000502,4.1,15,movie,Bohemios,Bohemios,0,1905,\N,100,\N
3,tt0000574,6.0,852,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
4,tt0000591,5.0,21,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama


In [ ]:
merged_data.shape

(298281, 11)

In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298281 entries, 0 to 298280
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          298281 non-null  object 
 1   averageRating   298281 non-null  float64
 2   numVotes        298281 non-null  int64  
 3   titleType       298281 non-null  object 
 4   primaryTitle    298281 non-null  object 
 5   originalTitle   298281 non-null  object 
 6   isAdult         298281 non-null  object 
 7   startYear       298281 non-null  object 
 8   endYear         298281 non-null  object 
 9   runtimeMinutes  298281 non-null  object 
 10  genres          298281 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 27.3+ MB


In [ ]:
# Convert data types and rename columns
merged_data['startYear'] = pd.to_numeric(merged_data['startYear'], errors='coerce')
merged_data = merged_data.rename(columns={'tconst': 'movieID', 'averageRating': 'rating', 'numVotes': 'votes'})
merged_data.head()

,movieID,rating,votes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,5.3,207,movie,Miss Jerry,Miss Jerry,0,1894.0,\N,45,Romance
1,tt0000147,5.3,482,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897.0,\N,100,"Documentary,News,Sport"
2,tt0000502,4.1,15,movie,Bohemios,Bohemios,0,1905.0,\N,100,\N
3,tt0000574,6.0,852,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906.0,\N,70,"Action,Adventure,Biography"
4,tt0000591,5.0,21,movie,The Prodigal Son,L'enfant prodigue,0,1907.0,\N,90,Drama


In [ ]:
# Data cleaning
merged_data = merged_data.dropna()

merged_data.shape

(298244, 11)

In [ ]:
merged_data.head()

,movieID,rating,votes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,5.3,207,movie,Miss Jerry,Miss Jerry,0,1894.0,\N,45,Romance
1,tt0000147,5.3,482,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897.0,\N,100,"Documentary,News,Sport"
2,tt0000502,4.1,15,movie,Bohemios,Bohemios,0,1905.0,\N,100,\N
3,tt0000574,6.0,852,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906.0,\N,70,"Action,Adventure,Biography"
4,tt0000591,5.0,21,movie,The Prodigal Son,L'enfant prodigue,0,1907.0,\N,90,Drama


### STEP 3 : Load Transfromed Data Into PostgreSQL.

Étape 1 : Configuration de l'environnement SQL Server

In [1]:
!pip install pyodbc

Étape 2 : Connexion à la base de données SQL Server

In [6]:
pyodbc.drivers()

['SQL Server',
 'Oracle dans OraDb11g_home1',
 'Oracle dans OraDb11g_home2',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'ODBC Driver 17 for SQL Server']

In [9]:
conn = pyodbc.connect(
    Trusted_Connection='yes',
    Driver='{ODBC Driver 17 for SQL Server}',
    Server='DESKTOP-E2V6SMS\SQLEXPRESS',
    Database='MovieRatingsDB'
)

cursor = conn.cursor()

In [ ]:
# Step 4: Load Data into the Database
for _, row in merged_data.iterrows():
    cursor.execute("INSERT INTO movies (movieID, title, startYear, genres) VALUES (?, ?, ?, ?)",
                   (row['movieID'], row['originalTitle'], row['startYear'], row['genres']))

for _, row in merged_data.iterrows():
    cursor.execute("INSERT INTO ratings (movieID, rating, votes) VALUES (?, ?, ?)",
                   (row['movieID'], row['rating'], row['votes']))
conn.commit()
cursor.close()
conn.close()